In [ ]:
%matplotlib inline
import math as math
import pandas as pd
import numpy as np
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt

In [ ]:
folderpath = "*"
# folderpath = "C:\\Users\\Benson\\Desktop"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft = pd.read_csv(file)
file = folderpath +  "\\lyftlong\\rand_scenes_table1.csv"
st = pd.read_csv(file)

In [ ]:
class Scene_Temporal_Info:
    def __init__(self, scene_table: pd.DataFrame):
        # NOTE: scene_table is already filtered to be only 1 scene
        self.scene_table = scene_table
        self.AV_locations = list(zip(list(self.scene_table['ego_translation_x']), 
                                     list(self.scene_table['ego_translation_y'])))
        self.velocities = []
        self.translations = []
        self.unit_dotprods = []
    
    def _unit(self, a: [float, float]) -> [float, float]:
        """Gets the unit of vector a. Returns zero vector if norm causes zero division
        Args:
            a ([float, float]): vector a
        Returns:
            float: the unit of vector a
        """
        norm = self._vector_norm(a)
        if norm == 0:
            return [0.0, 0.0]
        else:
            return [a_i/norm for a_i in a]
    def _vector_norm(self, a: [float]) -> float:
        """Gets the norm of vector a
        Args:
            a ([float, float]): vector a
        Returns:
            float: the norm of both vectors
        """
        return (sum([a_i**2 for a_i in a]))**(1/2)
    def _dotprod(self, a, b) -> float:
        """Gets the dotprod of vectors a and b
        Args:
            a ([float, float]): vector a
            b ([float, float]): vector b
        Returns:
            float: the dotprods of both vectors
        """
        return sum([ai*bi for ai, bi in zip(a, b)])
    def _translation(self, a: [float, float], b: [float, float]) -> [float, float]:
        """Gets the translation coords of points a and b
        Args:
            a ([float, float]): point a
            b ([float, float]): point b
        Returns:
            [float, float]: distance in x and distance in y between both points
        """
        ax, ay = a
        bx, by = b
        return [(bx - ax), (by - ay)]
    def set_velocities(self):
        velocities = []
        self.translations.clear()
        for i in range(1, len(self.AV_locations)):
            pointB = self.AV_locations[i]
            pointA = self.AV_locations[i - 1]
            side_length = self._translation(pointA, pointB)
            self.translations.append(side_length)
            velocity = self._vector_norm(side_length)
            velocities.append(velocity)
        self.velocities = velocities
        assert len(self.translations) == len(self.velocities)
        assert len(self.velocities)+1 == len(self.AV_locations)
        return velocities
    def set_unit_dotprods(self):
        if len(self.translations) == 0:
            self.set_velocities()
        unit_dotprods = []
        for i in range(1, len(self.translations)):
            vectB = self.translations[i]
            vectA = self.translations[i - 1]
            unit_dotprod = self._dotprod(self._unit(vectA), 
                                         self._unit(vectB))
            unit_dotprods.append(unit_dotprod)
        self.unit_dotprods = unit_dotprods
        return unit_dotprods        
    def get_unit_dotprods(self) -> [float]:
        assert self.unit_dotprods != []
        return [0.0, 0.0] + self.unit_dotprods
    def get_velocities(self) -> [float]:
        assert self.velocities != []
        return [0.0] + self.unit_dotprods
    def run(self):
        self.set_velocities()
        self.set_unit_dotprods()

In [ ]:
def scene_table_subset(scene_index: int, table: pd.DataFrame = ft) -> pd.DataFrame:
    scene_table = table[table.scene_index == scene_index]
    return scene_table
scene_indices = pd.unique(ft['scene_index'].values)


scene_indices = [ 7827, 15982, 10378,   154,  9380, 15002,  4455,  8285, 10522,
        2810,  1233,  1094,  3475, 13521,  2989,   385, 13242, 15073,
         721,  5601,  9884,  3818,  1372,  8029,  9520, 14617,  3043,
       13122, 13699, 12155, 15371,  5025,  1495,  8679, 13317,   204,
       14294, 11121, 13106, 14812,  8581,  9953,  5475,  8368,  1162,
       12928,  2595, 15782,   378,  3163, 15761,  2259, 15022, 15410,
         665,  4781, 10834, 12188,  1820, 12334, 15643,  2180,  2281,
        6235, 10847, 11405, 10210, 14560,  8968,  4386,  6303, 11197,
       14339,  2013,  9767, 13168, 11383, 14898,   296,  6343, 12055,
         112, 13893, 11414, 15559, 11667, 15367,  2547,   587, 11458,
       13215, 15416, 14090,  2367, 12803,  7992, 10719, 11029,  8566,
       15427]

In [ ]:
rand_index = np.random.choice(scene_indices)
scene_table = scene_table_subset(rand_index)
tmp_info = Scene_Temporal_Info(scene_table)
tmp_info.run()
print(rand_index)
# plt.ylim(-1, 1)
plt.plot(tmp_info.unit_dotprods)

In [ ]:
plt.plot(tmp_info.velocities) # in a given scene
# velocities over a given scene

In [ ]:
all_unit_dotprods = []
for scene_index in scene_indices:
    scene_table = scene_table_subset(scene_index)
    tmp_info = Scene_Temporal_Info(scene_table)
    tmp_info.run()
    all_unit_dotprods = all_unit_dotprods + tmp_info.unit_dotprods

In [ ]:
v2 = tmp_info.get_velocities()

In [ ]:
all_velocities = []
all_velocities_adjusted = []
for scene_index in scene_indices:
    scene_table = scene_table_subset(scene_index)
    tmp_info = Scene_Temporal_Info(scene_table)
    tmp_info.run()
    all_velocities_adjusted = all_velocities_adjusted + [tmp_info.velocities[0]] + tmp_info.velocities
    all_velocities = all_velocities + tmp_info.velocities

In [ ]:
dpr = pd.DataFrame()
dpr['dot_products'] = all_unit_dotprods

In [ ]:
def get_range(dp):
    intervals = [(-1.0, -0.8),
     (-0.8, -0.6),
     (-0.6, -0.4),
     (-0.4, -0.2),
     (-0.2, 0.0),
     (0.0, 0.2),
     (0.2, 0.4),
     (0.4, 0.6),
     (0.6, 0.8),
     (0.8, 1.0)]
    ranks = [-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8]
    for i in range(len(intervals)):
        intvl = intervals[i]
        if dp > intvl[0] and dp <= intvl[1]:
            return ranks[i]
        if dp > 1.0:
            return 0.8
        
dpr['range'] = [get_range(dp) for dp in all_unit_dotprods]

In [ ]:
plt.scatter(list(range(24637)), all_unit_dotprods, s=0.1)

In [ ]:
# plt.plot(all_velocities) # in a given scene
plt.scatter(list(range(24737)), all_velocities, s=0.1)

In [ ]:
all_unit_dotprods.__len__()

In [ ]:
dpr.sort_values('dot_products')

In [ ]:
dpr.groupby('range', sort=True).count()

dpr.to_csv("C:\\Users\\Benson\\Desktop\\lyftlong\\rand_norm_dotprods_table1.csv", index=False)

In [ ]:
# PREDICTING UNKNOWN TRAFFIC

In [ ]:
fta0 = ft['agent_index_interval_start']
fta1 = ft['agent_index_interval_end']

agents_amount = []
for i in range(len(ft)):
    a0i = fta0[i]
    a1i = fta1[i]
    agents_amount.append(a1i - a0i)

In [ ]:
plt.hist(agents_amount)
plt.title("Histogram of frequency of agent amounts")
plt.ylabel("Frequency of agent amount")
plt.xlabel("Agent amount for a given frame")

ft.to_csv("C:\\Users\\Benson\\Desktop\\lyftlong\\rand_frames_table1.csv", index=False)

In [ ]:
def get_mean_distance_agents(at = at):
        at_temp = at[['centroid_x','centroid_y','frame_index']]
        ft_temp = ft[['ego_translation_x', 'ego_translation_y', 'frame_index']]
        at_temp = at_temp.merge(ft_temp, on="frame_index")
        dx = (at_temp['centroid_x'] - at_temp['ego_translation_x'])**2
        dy = (at_temp['centroid_y'] - at_temp['ego_translation_y'])**2
        dist = (dx + dy)**(1/2)
        at_temp['dist'] = dist
        meandist = at_temp.groupby('frame_index',sort=False).mean()['dist']
        return meandist.values

mean_dist_values = get_mean_distance_agents(at )
ft['Mean Distance from Agents'] = mean_dist_values

In [ ]:
plt.scatter(list(range(len(ft))), ft['Mean Distance from Agents'], s=0.1)

In [ ]:
all_velocities_adjusted.__len__()

In [ ]:
plt.scatter(ft['Mean Distance from Agents'], all_velocities_adjusted, s=0.2)

In [ ]:
plt.hist(ft['Mean Distance from Agents'])

In [ ]:
# getting thershold radius

In [ ]:
frame_dotprods = []

for i in range(100):
    si = scene_indices[i]
    sft = ft[ft.scene_index == si]
    mowd = Scene_Temporal_Info(sft)
    mowd.run()
    dpi = mowd.get_unit_dotprods()
    frame_dotprods = frame_dotprods + dpi

In [ ]:
ft['dotprod'] = frame_dotprods

## Neural Network

In [ ]:
class Table_Aggregation:
    def __init__(self):
        pass
    def aggregator(self, table: pd.DataFrame = None) -> np.array:
        """ Collapses a table of a certain frame_index in a source table into an array
        Args:
            table (pd.DataFrame): the subset table of the source table with a certain frame_index
        Returns:
            np.array: the horizontal aggregation of the columns in the subsettted table
        """
        return None

In [ ]:
class Agent_Aggregation (Table_Aggregation):
    def __init__(self, table: pd.DataFrame = at, columns: [str] = []):
        super(Agent_Aggregation, self).__init__()
        self.table = table # agent_table
        self.columns = columns
        subset_table = table[columns].copy() #[table.frame_index == frame_index][columns].copy()
        self.subset_table = subset_table
    def aggregator(self, table: pd.DataFrame = None) -> pd.DataFrame:
        if table == None:
            table = self.subset_table 
        table['temp_index'] = table['frame_index']
        avgs = table.groupby('temp_index', sort=False).mean()
        return avgs

In [ ]:
at['size'] = at['extent_x'] * at['extent_y']

In [ ]:
at

In [ ]:
nn1 = Agent_Aggregation(table = at,
                        columns = [
                            "size",
                            "frame_index",
                            "velocity_x",
                            "velocity_y",
                            "yaw"
                        ])

In [ ]:
nn1_table = nn1.aggregator()

In [ ]:
nn1_table

In [ ]:
ft2 = ft.merge(nn1_table)

In [ ]:
ft2

In [ ]:
class Frame_Instance:
    def __init__(self, frame_0_array: np.array, frame_1_transl: np.array):
        self.frame_0_array = frame_0_array
        self.frame_1_translation = frame_1_transl

In [ ]:
unique_scenes_indices = pd.unique(ft2['scene_index'])

In [ ]:
frame_instances = []

In [ ]:
def frame_encoder(frame_0_row):
    reverse = frame_0_row['dotprod']
    frame_0_row = frame_0_row.drop(['dotprod',
    'timestamp', 
    'agent_index_interval_start', 
    'agent_index_interval_end',
    'traffic_lights_start', 
    'traffic_lights_end',
    'frame_index', 'scene_index'])
    input_ = np.array(frame_0_row)
    if reverse < 0:
        label = 0
    if reverse >= 0:
        label = 1
    assert len(input_) == 17
    return input_, label

In [ ]:
for i in range(len(ft2)):
    row = ft2.iloc[i]
    input_, label = frame_encoder(row)
    frame_instance = Frame_Instance(input_, label)
    frame_instances.append(frame_instance)

In [ ]:
X = []
y = []
for frame_instance in frame_instances:
    X.append(list(frame_instance.frame_0_array))
    if frame_instance.frame_1_translation == 0:
        y.append(0)
    else:
        y.append(1)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from collections import Counter

In [ ]:
counter = Counter(y)
print(counter)

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
counter = Counter(y)
print(counter)

In [ ]:
new_frame_instances = []
for i in range(len(X)):
    input_ = X[i]
    label = y[i]
    frame_instance = Frame_Instance(input_, label)
    new_frame_instances.append(frame_instance)

In [ ]:
new_frame_instances = np.array(new_frame_instances)

In [ ]:
np.random.shuffle(new_frame_instances)

In [ ]:
TRAIN_PROPORTION = 0.8
TRAIN_PROPORTION_INDEXER = int(len(new_frame_instances) * TRAIN_PROPORTION)
nrevtr = new_frame_instances[:TRAIN_PROPORTION_INDEXER]
nrevt = new_frame_instances[TRAIN_PROPORTION_INDEXER:]

In [ ]:
#TRAIN_PROPORTION_INDEXER len(new_frame_instances)
len(nrevtr)
9912 

In [ ]:
len(new_frame_instances)
np_frame_instances[:]

In [ ]:
Train_Dataset = np.array(nrevtr)
Test_Dataset = np.array(nrevt)
np.random.shuffle(Train_Dataset)
np.random.shuffle(Test_Dataset)

In [ ]:
train_x = []
train_y = []
for frame_instance in Train_Dataset:
    train_xi = frame_instance.frame_0_array
    train_x.append(train_xi)
    train_yi = frame_instance.frame_1_translation
    train_y.append(train_yi)

In [ ]:
test_x = []
test_y = []
for frame_instance in Test_Dataset:
    test_xi = frame_instance.frame_0_array
    test_x.append(test_xi)
    test_yi = frame_instance.frame_1_translation
    test_y.append(test_yi)

# test_x = [frame_instance.frame_0_array for frame_instance in Test_Dataset]
# test_y = [frame_instance.frame_1_translation for frame_instance in Test_Dataset]

In [ ]:
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)


In [ ]:
import keras
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense

In [ ]:
model = tf.keras.Sequential()
model.add(Dense(16, input_shape=(17,)))
model.add(Dense(8))
model.add(Dense(4))
model.add(Dense(2))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='mean_squared_error')

In [ ]:
model.fit(train_x, train_y, epochs = 10)

In [ ]:
successes = 0
for i in range(len(test_y)):
    if i % 2000 == 0 and i != 0:
        print("{0} of {1} samples tested".format(i, len(test_y)))
    pred = model.predict(np.array([test_x[i],]))[0][0]
    target = (test_y[i])
    if target >= .5:
        target == 1
    else:
        target == 0
    if target == pred:
        successes += 1

In [ ]:
successes/len(test_y)